In [2]:
import numpy as np

# 构造旋转矩阵（使用scipy）
def construct_rotation_matrix():
    """构造旋转矩阵"""
    # 手动生成一个旋转矩阵（绕各轴旋转一定角度）
    # 绕x轴旋转30度
    theta_x = np.radians(30)
    R_x = np.array([[1, 0, 0],
                    [0, np.cos(theta_x), -np.sin(theta_x)],
                    [0, np.sin(theta_x), np.cos(theta_x)]])
    
    # 绕y轴旋转45度
    theta_y = np.radians(45)
    R_y = np.array([[np.cos(theta_y), 0, np.sin(theta_y)],
                    [0, 1, 0],
                    [-np.sin(theta_y), 0, np.cos(theta_y)]])
    
    # 绕z轴旋转60度
    theta_z = np.radians(60)
    R_z = np.array([[np.cos(theta_z), -np.sin(theta_z), 0],
                    [np.sin(theta_z), np.cos(theta_z), 0],
                    [0, 0, 1]])
    
    # 组合旋转矩阵 R = R_z * R_y * R_x
    R_matrix = R_z @ R_y @ R_x
    
    print("生成的旋转矩阵:")
    print(R_matrix)
    print()
    
    return R_matrix

# 验证旋转矩阵
def validate_rotation_matrix(R_matrix):
    """验证旋转矩阵"""
    # 检查行列式是否为1
    det = np.linalg.det(R_matrix)
    
    # 检查是否为正交矩阵
    is_orthogonal = np.allclose(R_matrix @ R_matrix.T, np.eye(3))
    
    return det, is_orthogonal

# 构造平移向量的反对称矩阵
def skew_symmetric(t):
    """构造平移向量的反对称矩阵"""
    return np.array([
        [0, -t[2], t[1]],
        [t[2], 0, -t[0]],
        [-t[1], t[0], 0]
    ])

# 模拟本质矩阵
def simulate_essential_matrix():
    """模拟本质矩阵"""
    # 构造旋转矩阵
    R_matrix = construct_rotation_matrix()
    
    # 验证旋转矩阵
    det, is_orthogonal = validate_rotation_matrix(R_matrix)
    
    print("旋转矩阵验证:")
    print(f"行列式: {det:.6f}")
    print(f"是否为正交矩阵: {is_orthogonal}")
    
    # 定义平移向量
    t = np.array([0.1, 0.2, -0.4])
    print(f"\n平移向量: {t}")
    
    # 构造平移向量的反对称矩阵
    t_skew = skew_symmetric(t)
    
    # 计算本质矩阵 E = [t]_× * R
    E = t_skew @ R_matrix
    print(f"\n本质矩阵 E:\n{E}")
    
    return E, t, R_matrix

# 对本质矩阵进行SVD分解
def svd_for_e(E, t):
    """对本质矩阵进行SVD分解"""
    print("\n对本质矩阵进行SVD分解:")
    
    # SVD分解
    U, singular_values, VT = np.linalg.svd(E)
    
    print(f"U矩阵:\n{U}")
    print(f"\n奇异值: {singular_values}")
    print(f"\nV^T矩阵:\n{VT}")
    
    # 验证本质矩阵的奇异值特性
    print("\n验证本质矩阵的奇异值特性:")
    print(f"前两个奇异值是否近似相等: {np.isclose(singular_values[0], singular_values[1])}")
    print(f"第三个奇异值是否近似为0: {np.isclose(singular_values[2], 0)}")
    
    # 从SVD结果重构旋转矩阵和平移向量
    print("\n从SVD结果重构旋转矩阵和平移向量:")
    
    # 构造W矩阵
    W = np.array([
        [0, -1, 0],
        [1, 0, 0],
        [0, 0, 1]
    ])
    
    # 可能的两种旋转矩阵解
    R1 = U @ W @ VT
    R2 = U @ W.T @ VT
    
    print(f"\n第一种可能的旋转矩阵 R1:\n{R1}")
    print(f"\n第二种可能的旋转矩阵 R2:\n{R2}")
    
    # 估计的平移向量（注意：只能确定方向，不能确定尺度）
    t_estimated = U[:, 2]
    print(f"\n估计的平移方向（U的第三列）: {t_estimated}")
    
    t_normalized = t / np.linalg.norm(t)
    print(f"原始平移向量的方向: {t_normalized}")
    
    # 多种方法检查两者是否平行
    print("\n平行性验证:")
    
    # 方法1: 点积绝对值是否接近1
    dot_product = np.dot(t_estimated, t_normalized)
    parallel_by_dot = np.isclose(np.abs(dot_product), 1.0)
    print(f"方法1 - 点积绝对值接近1: {parallel_by_dot} (点积={dot_product:.6f})")
    
    # 方法2: 叉积是否接近0向量
    cross_product = np.cross(t_estimated, t_normalized)
    cross_norm = np.linalg.norm(cross_product)
    parallel_by_cross = np.isclose(cross_norm, 0.0)
    print(f"方法2 - 叉积接近0向量: {parallel_by_cross} (叉积模长={cross_norm:.6f})")
    
    # 方法3: 夹角是否接近0度或180度
    cos_angle = dot_product / (np.linalg.norm(t_estimated) * np.linalg.norm(t_normalized))
    cos_angle = np.clip(np.abs(cos_angle), -1.0, 1.0)  # 确保数值稳定
    angle_rad = np.arccos(cos_angle)
    angle_deg = np.degrees(angle_rad)
    parallel_by_angle = np.isclose(angle_deg, 0.0) or np.isclose(angle_deg, 180.0)
    print(f"方法3 - 夹角接近0°或180°: {parallel_by_angle} (夹角={angle_deg:.6f}°)")
    
    # 综合判断
    is_parallel = parallel_by_dot and parallel_by_cross and parallel_by_angle
    print(f"\n综合判断 - 向量是否平行: {is_parallel}")
    
    return U, singular_values, VT

# 主函数
def main():
    """主函数"""
    print("验证本质矩阵")
    
    # 模拟本质矩阵
    E, t, R_matrix = simulate_essential_matrix()
    
    # 对本质矩阵进行SVD分解
    U, singular_values, VT = svd_for_e(E, t)
    
    print("\n验证完成")

if __name__ == "__main__":
    main()

验证本质矩阵
生成的旋转矩阵:
[[ 0.35355339 -0.5732233   0.73919892]
 [ 0.61237244  0.73919892  0.28033009]
 [-0.70710678  0.35355339  0.61237244]]

旋转矩阵验证:
行列式: 1.000000
是否为正交矩阵: True

平移向量: [ 0.1  0.2 -0.4]

本质矩阵 E:
[[ 0.10352762  0.36639025  0.23460652]
 [-0.07071068  0.19393398 -0.35691681]
 [-0.00947343  0.18856455 -0.11980678]]

对本质矩阵进行SVD分解:
U矩阵:
[[ 0.52379345  0.82342053  0.21821789]
 [ 0.70229804 -0.56240668  0.43643578]
 [ 0.48209738 -0.07534821 -0.87287156]]

奇异值: [4.58257569e-01 4.58257569e-01 1.42504091e-17]

V^T矩阵:
[[-0.          0.91437386 -0.40487089]
 [ 0.27436258  0.3893345   0.87928597]
 [ 0.96162632 -0.11108142 -0.25086998]]

验证本质矩阵的奇异值特性:
前两个奇异值是否近似相等: True
第三个奇异值是否近似为0: True

从SVD结果重构旋转矩阵和平移向量:

第一种可能的旋转矩阵 R1:
[[ 0.06613474  0.5247434  -0.84868755]
 [ 0.22700383 -0.83615873 -0.49930735]
 [-0.97164575 -0.15963376 -0.1744179 ]]

第二种可能的旋转矩阵 R2:
[[ 0.35355339 -0.5732233   0.73919892]
 [ 0.61237244  0.73919892  0.28033009]
 [-0.70710678  0.35355339  0.61237244]]

估计的平移方向（U的第三列）: [ 0